In [1]:
import requests
url = "https://herp.careers/careers/jobs"
resp = requests.get(url, timeout=15)
print("Status Code: ", resp.status_code)
print(resp.text[:1000])

Status Code:  200
<!DOCTYPE html><html lang="ja" class="h-full scroll-smooth"><head prefix="og: https://ogp.me/ns#"><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"/><link rel="stylesheet" href="/_next/static/css/d883a9987447c9ae.css" data-precedence="next"/><link rel="preload" as="script" fetchPriority="low" href="/_next/static/chunks/webpack-e8863feb85785555.js"/><script src="/_next/static/chunks/0717c08b-9e47371bac49531b.js" async=""></script><script src="/_next/static/chunks/292-87e4cff6abef8e16.js" async=""></script><script src="/_next/static/chunks/main-app-f9d77102cc7c9099.js" async=""></script><script src="/_next/static/chunks/4675-aecea3eb8c35b13d.js" async=""></script><script src="/_next/static/chunks/5575-99482ebddf14fb37.js" async=""></script><script src="/_next/static/chunks/7798-acce0724fcd725b8.js" async=""></script><script src="/_next/static/chunks/8196-cc43a7421136b77d.js" async=""></script><script src="/_next/

In [5]:
#Cell2: BeautifulSoupでHTMLをパースし、求人タイトルを抽出
from bs4 import BeautifulSoup
url = "https://herp.careers/careers/jobs"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# 求人タイトルを抽出（クラス名の一部で抽出）
job_titles = [
    el.get_text(strip=True)
    for el in soup.find_all("h3")
    if el.get("class") and any("text-[20px]" in c for c in el.get("class"))
]

# 最初の20件を表示
job_titles[:20] 

['株式会社LESS PLUS DESIGN',
 '株式会社ハイウェイ',
 '株式会社Art to Heritage',
 'A1A株式会社',
 'ココザス株式会社',
 '株式会社Conoris Technologies',
 'ROSCA株式会社',
 '株式会社シンカ',
 'REHATCH株式会社',
 '株式会社HACHI',
 'Nature株式会社',
 'エルピクセル株式会社',
 '株式会社GENBA',
 '株式会社Alumnote',
 'NE株式会社',
 '株式会社LabBase',
 'SHE株式会社',
 'クレジットエンジン株式会社',
 'リンクタイズホールディングス株式会社',
 'ちゅらデータ株式会社']

In [3]:
# Cell3: 表示されている1ページ目のみから (company_name, company_url) の組を抽出
import urllib.parse
base_url = "https://herp.careers"
company_pairs = []
seen = set()
# 前のセルで作成した soup (BeautifulSoup オブジェクト) を前提とする
for h3 in soup.find_all('h3'):
    classes = h3.get('class') or []
    if not any('text-[20px]' in c for c in classes):
        continue
    company_name = h3.get_text(strip=True)
    parent_a = h3.find_parent('a', href=True)
    if not parent_a:
        continue
    href = parent_a.get('href')
    if not href:
        continue
    company_url = urllib.parse.urljoin(base_url, href)
    # 企業ページのみを対象にし、重複を除く
    if '/careers/companies/' not in company_url:
        continue
    if company_url in seen:
        continue
    seen.add(company_url)
    company_pairs.append({'company': company_name, 'company_url': company_url})

print(f'Found {len(company_pairs)} unique company pairs on the first page')
company_pairs[:20]
# Check Cell3 output: prints count and sample pairs
try:
    print('Found', len(company_pairs), 'company pairs on the first page')
    company_pairs[:20]
except NameError:
    print('company_pairs not defined yet; run the previous cells to create soup and company_pairs')

print('Company name and URL: ', company_pairs)

Found 20 unique company pairs on the first page
Found 20 company pairs on the first page
Company name and URL:  [{'company': '株式会社LESS PLUS DESIGN', 'company_url': 'https://herp.careers/careers/companies/lpdcoltd?'}, {'company': '株式会社ハイウェイ', 'company_url': 'https://herp.careers/careers/companies/hiway?'}, {'company': '株式会社Art to Heritage', 'company_url': 'https://herp.careers/careers/companies/artheritage?'}, {'company': 'A1A株式会社', 'company_url': 'https://herp.careers/careers/companies/a1aco?'}, {'company': 'ココザス株式会社', 'company_url': 'https://herp.careers/careers/companies/cocozas?'}, {'company': '株式会社Conoris Technologies', 'company_url': 'https://herp.careers/careers/companies/mitsucaru?'}, {'company': 'ROSCA株式会社', 'company_url': 'https://herp.careers/careers/companies/rosca?'}, {'company': '株式会社シンカ', 'company_url': 'https://herp.careers/careers/companies/thinca?'}, {'company': 'REHATCH株式会社', 'company_url': 'https://herp.careers/careers/companies/rehatch?'}, {'company': '株式会社HACHI', '

In [6]:
# Cell4: 2ページ目のみから (company_name, company_url) の組を抽出
import requests
from bs4 import BeautifulSoup
import urllib.parse

url_page2 = "https://herp.careers/careers/jobs?page=2"
resp2 = requests.get(url_page2, timeout=15)
resp2.raise_for_status()
soup2 = BeautifulSoup(resp2.text, 'html.parser')

company_pairs_page2 = []
seen2 = set()
for h3 in soup2.find_all('h3'):
    classes = h3.get('class') or []
    if not any('text-[20px]' in c for c in classes):
        continue
    company_name = h3.get_text(strip=True)
    parent_a = h3.find_parent('a', href=True)
    if not parent_a:
        continue
    href = parent_a.get('href')
    if not href:
        continue
    company_url = urllib.parse.urljoin('https://herp.careers', href)
    if '/careers/companies/' not in company_url:
        continue
    if company_url in seen2:
        continue
    seen2.add(company_url)
    company_pairs_page2.append({'company': company_name, 'company_url': company_url})

print(f'Found {len(company_pairs_page2)} unique company pairs on page 2')
company_pairs_page2[:20]

Found 20 unique company pairs on page 2


[{'company': '株式会社BuySell Technologies',
  'company_url': 'https://herp.careers/careers/companies/buyselltech?'},
 {'company': '株式会社Asobica',
  'company_url': 'https://herp.careers/careers/companies/asobica?'},
 {'company': 'コネクテッドロボティクス株式会社',
  'company_url': 'https://herp.careers/careers/companies/crinc?'},
 {'company': '株式会社KINS',
  'company_url': 'https://herp.careers/careers/companies/yourkins?'},
 {'company': 'イタンジ株式会社',
  'company_url': 'https://herp.careers/careers/companies/itandi?'},
 {'company': '株式会社Magic Shields',
  'company_url': 'https://herp.careers/careers/companies/magicshields?'},
 {'company': '株式会社GenerativeX',
  'company_url': 'https://herp.careers/careers/companies/generativex?'},
 {'company': '株式会社UMITO',
  'company_url': 'https://herp.careers/careers/companies/umito?'},
 {'company': '株式会社HUMAN LIFE',
  'company_url': 'https://herp.careers/careers/companies/humanlife?'},
 {'company': '株式会社ビットエー',
  'company_url': 'https://herp.careers/careers/companies/bita?'},
 